# TASK 1
### Step 1

In [2]:
import pandas as pd

def load_data(file_path):
    return pd.read_csv(file_path)

testdata = load_data('adult_test.csv')
traindata = load_data('adult_train.csv')

Ytest = testdata['target']
Ytrain = traindata['target']
Xtest = testdata.drop('target', axis=1)
Xtrain = traindata.drop('target', axis=1)

### Step 2

In [3]:
Xtrain_dict = Xtrain.to_dict('records')
Xtest_dict = Xtest.to_dict('records')

from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer()

Xtrain_encoded = dv.fit_transform(Xtrain_dict)
Xtest_encoded = dv.transform(Xtest_dict)

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

clf1 = DecisionTreeClassifier(random_state=0, max_depth=7)
clf2 = RandomForestClassifier(random_state=0)
clf3 = GradientBoostingClassifier(random_state=0)
clf4 = Perceptron(random_state=0)
clf5 = LogisticRegression(random_state=0, max_iter=2000)
clf6 = LinearSVC(random_state=0, max_iter=2000)
clf7 = MLPClassifier(random_state=0, max_iter=500)

classifiers = [clf1, clf2, clf3, clf4, clf5, clf6, clf7]
classifier_names = ['Decision Tree', 'Random Forest', 'Gradient Boosting', 'Perceptron',
                    'Logistic Regression', 'Linear SVC', 'MLP Classifier']

from sklearn.model_selection import cross_val_score
import numpy as np

# Used to compare classifiers
def compare_classifiers(Xtrain_encoded, Ytrain, classifiers, classifier_names):
    for clf, name in zip(classifiers, classifier_names):
        crossArray = cross_val_score(clf, Xtrain_encoded, Ytrain)
        score = np.mean(crossArray)
        print(f'{name} average cross-validation score: {score}')
    
compare_classifiers(Xtrain_encoded, Ytrain, classifiers, classifier_names)    

from sklearn.metrics import accuracy_score

# Gradient Boosting Classifier selected based on the best cross-validation score
clf3.fit(Xtrain_encoded, Ytrain)
Yguess = clf3.predict(Xtest_encoded)

print(f"Final Test Accuracy: {accuracy_score(Ytest, Yguess)}")

Decision Tree average cross-validation score: 0.8555020227924419
Random Forest average cross-validation score: 0.8467491260155932
Gradient Boosting average cross-validation score: 0.8656675592304335
Perceptron average cross-validation score: 0.7660086178050249


c:\Users\smelk\ChalmersCode\AML (DAT341)\Applied-Machine-Learning-DAT341-\sklearn-env\Lib\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 2000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=2000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\smelk\ChalmersCode\AML (DAT341)\Applied-Machine-Learning-DAT341-\sklearn-env\Lib\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 2000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=2000).
Yo

Logistic Regression average cross-validation score: 0.8509260396236444
Linear SVC average cross-validation score: 0.8515094975424317
MLP Classifier average cross-validation score: 0.8485305093838027
Final Test Accuracy: 0.871199557766722


### Step 3

In [4]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
  
pipeline = make_pipeline(
    DictVectorizer(), # Convert data to correct format
    StandardScaler(with_mean=False), # Scales features to better suit the model
    SelectKBest(f_classif, k=50), # Selects the best 50 features based on f_classif
    GradientBoostingClassifier() # Our chosen classifier
)

pipeline.fit(Xtrain_dict, Ytrain)
Yguess_pipeline = pipeline.predict(Xtest_dict)
print(f"Final Test Accuracy with Pipeline: {accuracy_score(Ytest, Yguess_pipeline)}")

Final Test Accuracy with Pipeline: 0.8699097107057306


The pipeline did get a bit worse results however this is due to the f_classif evaluator being meant for numerical features but our data does have class based data as well. Seperating what test to use on what feature is possible if we would've used ColumnTransformer instead of DictVectorizer.